In [ ]:
import cv2
import glob
import numpy as np
import pandas as pd
import skimage
from skimage import measure
from skimage import morphology
from skimage import segmentation

In [ ]:
segment_biposy_images()

In [ ]:
counter = 0

def segment_biopsy_images():

    for grade_no in range(0, 5):
        
        src_path = "G:/LiverCancer/original/" + "grade" + str(grade_no) + "/"
        src_files = glob.glob(src_path + "*.jpg")
    
        for src_file in src_files:

            print(src_file)
            
            segment_biopsy_image(src_file)
        
def segment_biopsy_image(src_file):

    dst_path = "G:/LiverCancer/nucleus/_labels/"

    global counter; labels = []

    sample_image = cv2.imread(file_path)
    image_red = sample_image[:,:,2]
    kernel_1 = skimage.morphology.disk(20)
    kernel_2 = skimage.morphology.disk(3)

    image_closing = skimage.morphology.closing(image_red, kernel_2)
    image_erosion = skimage.morphology.erosion(image_closing, kernel_1)
    image_reconstruction_1 = skimage.morphology.reconstruction(image_erosion,image_closing).astype(np.uint8)
    image_dialtion = skimage.morphology.dilation(image_reconstruction_1, kernel_1)
    image_reconstruction_2 = skimage.morphology.reconstruction(image_dialtion, image_reconstruction_1, method='erosion').astype(np.uint8)

    image_zero = (image_reconstruction_2 - image_reconstruction_2.mean()).astype(np.int)
    image_binary = image_zero < - 10
    image_binary = skimage.segmentation.clear_border(image_binary)
    image_label = skimage.measure.label(image_binary)
    image_label_color = skimage.color.label2rgb(image_label) 
    print('regions number:',image_label.max()+1)
    properties = skimage.measure.regionprops(image_label)

    for i in range(len(properties)):

        coord_x = int(properties[i].centroid[0])
        coord_y = int(properties[i].centroid[1])
        diameter = int(properties[i].equivalent_diameter)

        labels.append([file_path, i, coord_x, coord_y, diameter])

    df_labels = pd.DataFrame(labels, columns=["file_path", "index", "coord_x", "coord_y", "diameter"])
    
    df_labels.to_csv(dst_path + str(counter) + ".csv", index=False)

    counter += 1
